# Lab 4: Capstone - Build Your Own Agent

**Objective:** Apply everything you've learned to build a complete, functional AI agent.

**Duration:** ~45 minutes

**What You'll Do:**
- Choose an agent type and framework
- Design and implement your agent
- Test and refine your agent
- Prepare a brief demo

## Part 1: Choose Your Project

Select one of the following agent types (or propose your own!):

### Option A: Code Assistant Agent
An agent that helps developers with code-related tasks.

**Capabilities:**
- Analyze code for bugs or issues
- Explain what code does
- Suggest improvements
- Generate documentation

### Option B: Research Agent
An agent that conducts research on topics.

**Capabilities:**
- Search for information
- Synthesize findings from multiple sources
- Create structured summaries
- Save and organize notes

### Option C: Data Analyst Agent
An agent that helps analyze and understand data.

**Capabilities:**
- Load and process CSV/JSON data
- Calculate statistics
- Identify patterns and trends
- Create visualizations

### Option D: Your Own Idea!
Build an agent for a use case you care about.

## Framework Selection Guide

**Not sure which framework to use?** Here's a quick decision helper:

| If your agent needs... | Use This Framework |
|------------------------|-------------------|
| Simple tool use (search, calculate, etc.) | **LangChain** |
| Multiple specialized agents collaborating | **AutoGen** or **CrewAI** |
| Complex stateful workflows with routing | **LangGraph** |
| Human approval at key decision points | **LangGraph** |
| Agents that debate/critique each other | **AutoGen** |
| Role-based task pipelines | **CrewAI** |

**Quick Start Recommendations:**
- **Code Assistant** → LangChain (simple tool use) or LangGraph (if you want code review workflow)
- **Research Agent** → LangChain (solo) or AutoGen/CrewAI (team of researcher + analyst + writer)
- **Data Analyst** → LangChain (straightforward) or LangGraph (multi-step pipeline)
- **Custom Idea** → Start with LangChain, upgrade to LangGraph if you need state management

In [ ]:
# Document your choice
project_choice = {
    "agent_type": "",  # A, B, C, or D
    "description": "",  # What does your agent do?
    "framework": "",  # LangChain, AutoGen, CrewAI, or LangGraph
    "why_this_framework": "",  # Why did you choose this framework?
    "tools_needed": [],  # What tools will your agent need?
    "success_criteria": ""  # How will you know it works?
}

print("Fill in your project choice above!")

## Part 2: Setup

Import the libraries you need based on your chosen framework.

In [ ]:
# Environment setup
import os
from dotenv import load_dotenv

load_dotenv()
assert os.getenv("OPENAI_API_KEY"), "OPENAI_API_KEY not found"
print("Environment configured!")

In [ ]:
# Import based on your framework choice

# LangChain imports
from langchain_openai import ChatOpenAI
from langchain.agents import AgentExecutor, create_react_agent
from langchain.tools import tool
from langchain import hub
from langchain.memory import ConversationBufferMemory

# AutoGen imports (uncomment if using)
# from autogen import AssistantAgent, UserProxyAgent, GroupChat, GroupChatManager

# CrewAI imports (uncomment if using)
# from crewai import Agent, Task, Crew, Process

# LangGraph imports (uncomment if using)
# from typing import TypedDict, Annotated
# from langgraph.graph import StateGraph, END, START
# from langgraph.checkpoint.memory import MemorySaver
# import operator

print("Imports complete!")

## Part 3: Define Your Tools

Create the tools your agent needs.

In [ ]:
# Example tools - modify or replace based on your agent type

# Code Assistant Tools
@tool
def analyze_code(code: str) -> str:
    """Analyze code and identify potential issues.
    
    Args:
        code: The code to analyze
    """
    # TODO: Implement actual analysis
    return f"Analyzing code ({len(code)} characters)...\nAnalysis complete."

# Research Tools
@tool
def search_web(query: str) -> str:
    """Search the web for information.
    
    Args:
        query: The search query
    """
    # TODO: Implement actual search (use Tavily if available)
    return f"Search results for '{query}': Found 5 relevant results."

@tool
def save_note(title: str, content: str) -> str:
    """Save a research note.
    
    Args:
        title: The note title
        content: The note content
    """
    # TODO: Implement actual storage
    return f"Note saved: {title}"

# Data Analyst Tools
@tool
def calculate_stats(data: str) -> str:
    """Calculate statistics on numerical data.
    
    Args:
        data: Comma-separated numbers
    """
    try:
        numbers = [float(x.strip()) for x in data.split(',')]
        avg = sum(numbers) / len(numbers)
        return f"Count: {len(numbers)}, Sum: {sum(numbers)}, Average: {avg:.2f}"
    except:
        return "Error: Could not parse data"

print("Sample tools defined!")
print("Modify or add tools based on your agent type.")

In [ ]:
# TODO: Define your custom tools here
# @tool
# def my_custom_tool(param: str) -> str:
#     """Description of what this tool does."""
#     return "result"

## Part 4: Build Your Agent

Implement your agent using your chosen framework.

In [ ]:
# LangChain Implementation Template - COMPLETE WORKING EXAMPLE

def build_langchain_agent():
    """Build a LangChain ReAct agent with custom tools."""
    
    # Initialize LLM
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
    
    # Define tools - customize these for your use case
    tools = [analyze_code, search_web, save_note, calculate_stats]
    
    # Get the ReAct prompt template
    prompt = hub.pull("hwchase17/react")
    
    # Create the agent
    agent = create_react_agent(llm, tools, prompt)
    
    # Create executor with memory for conversation history
    memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
    
    executor = AgentExecutor(
        agent=agent,
        tools=tools,
        memory=memory,
        verbose=True,  # Set to True to see agent reasoning
        max_iterations=10,
        handle_parsing_errors=True
    )
    
    return executor

# Build and test the LangChain agent
print("Building LangChain agent...")
my_agent = build_langchain_agent()
print("✅ LangChain agent built successfully!")

# Quick test
print("\n--- Quick Test ---")
result = my_agent.invoke({"input": "Calculate the average of 10, 20, 30, 40, 50"})
print(f"Result: {result['output']}")

In [ ]:
# AutoGen Implementation Template - COMPLETE WORKING EXAMPLE

from autogen import AssistantAgent, UserProxyAgent, GroupChat, GroupChatManager

def build_autogen_team():
    """Build an AutoGen multi-agent team."""
    
    llm_config = {
        "config_list": [{
            "model": "gpt-4o-mini",
            "api_key": os.getenv("OPENAI_API_KEY")
        }],
        "temperature": 0.7
    }
    
    # Define specialized agents
    researcher = AssistantAgent(
        name="Researcher",
        system_message="""You are a research specialist. Your job is to:
        - Search for and gather relevant information
        - Verify facts and sources
        - Summarize findings clearly
        When you have gathered enough information, say 'Research complete.'""",
        llm_config=llm_config
    )
    
    analyst = AssistantAgent(
        name="Analyst",
        system_message="""You are a data analyst. Your job is to:
        - Analyze information provided by the researcher
        - Identify patterns and insights
        - Make recommendations based on data
        When your analysis is complete, say 'Analysis complete.'""",
        llm_config=llm_config
    )
    
    writer = AssistantAgent(
        name="Writer",
        system_message="""You are a content writer. Your job is to:
        - Take research and analysis and create clear, well-structured content
        - Ensure the writing is accessible and engaging
        - Format output appropriately
        When the final content is ready, say 'TERMINATE'.""",
        llm_config=llm_config
    )
    
    # User proxy to initiate conversations
    user_proxy = UserProxyAgent(
        name="User",
        human_input_mode="NEVER",  # Set to "ALWAYS" for human-in-loop
        max_consecutive_auto_reply=0,
        code_execution_config=False
    )
    
    # Create group chat
    groupchat = GroupChat(
        agents=[user_proxy, researcher, analyst, writer],
        messages=[],
        max_round=10
    )
    
    manager = GroupChatManager(groupchat=groupchat, llm_config=llm_config)
    
    return user_proxy, manager

# Build and test the AutoGen team
print("Building AutoGen team...")
user_proxy, manager = build_autogen_team()
print("✅ AutoGen team built successfully!")

# Quick test (uncomment to run - uses API credits)
# print("\n--- Quick Test ---")
# user_proxy.initiate_chat(
#     manager,
#     message="Research the benefits of microservices architecture and write a brief summary."
# )

In [ ]:
# CrewAI Implementation Template - COMPLETE WORKING EXAMPLE

from crewai import Agent, Task, Crew, Process

def build_crewai_crew():
    """Build a CrewAI crew with specialized agents."""
    
    # Define agents with clear roles
    researcher = Agent(
        role="Research Specialist",
        goal="Find accurate, relevant information on the given topic",
        backstory="""You are an experienced researcher with a keen eye for detail.
        You excel at finding reliable sources and synthesizing information.""",
        verbose=True,
        allow_delegation=False
    )
    
    analyst = Agent(
        role="Data Analyst",
        goal="Analyze research findings and extract meaningful insights",
        backstory="""You are a skilled analyst who can see patterns others miss.
        You provide clear, actionable insights based on data.""",
        verbose=True,
        allow_delegation=False
    )
    
    writer = Agent(
        role="Content Writer",
        goal="Create clear, engaging content from research and analysis",
        backstory="""You are a talented writer who makes complex topics accessible.
        You create well-structured, professional content.""",
        verbose=True,
        allow_delegation=False
    )
    
    # Define tasks
    research_task = Task(
        description="""Research the given topic: {topic}
        
        Gather key facts, recent developments, and important context.
        Provide a structured summary of your findings.""",
        expected_output="A structured research summary with key facts and sources",
        agent=researcher
    )
    
    analysis_task = Task(
        description="""Analyze the research findings.
        
        Identify key patterns, trends, and insights.
        Provide recommendations based on the analysis.""",
        expected_output="Analysis with insights and recommendations",
        agent=analyst,
        context=[research_task]  # Uses output from research_task
    )
    
    writing_task = Task(
        description="""Create a final report from the research and analysis.
        
        Structure the content clearly with sections and bullet points.
        Make it accessible to a general audience.""",
        expected_output="A well-structured final report",
        agent=writer,
        context=[research_task, analysis_task]
    )
    
    # Create crew
    crew = Crew(
        agents=[researcher, analyst, writer],
        tasks=[research_task, analysis_task, writing_task],
        process=Process.sequential,
        verbose=True
    )
    
    return crew

# Build and test the CrewAI crew
print("Building CrewAI crew...")
my_crew = build_crewai_crew()
print("✅ CrewAI crew built successfully!")

# Quick test (uncomment to run - uses API credits)
# print("\n--- Quick Test ---")
# result = my_crew.kickoff(inputs={"topic": "benefits of microservices architecture"})
# print(f"Result: {result}")

In [ ]:
# LangGraph Implementation Template - COMPLETE WORKING EXAMPLE

from typing import TypedDict, Annotated, Literal
from langgraph.graph import StateGraph, END, START
from langgraph.checkpoint.memory import MemorySaver
import operator

def build_langgraph_agent():
    """Build a LangGraph workflow agent with state management."""
    
    # Define state schema
    class AgentState(TypedDict):
        messages: Annotated[list, operator.add]  # Accumulates messages
        task: str
        analysis: str
        result: str
        status: str
    
    # Initialize LLM
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
    
    # Define nodes
    def analyze_task(state: AgentState) -> dict:
        """Analyze the input task and determine approach."""
        task = state.get("task", "")
        response = llm.invoke(f"Analyze this task and describe the approach: {task}")
        return {
            "messages": [f"Analysis: {response.content}"],
            "analysis": response.content,
            "status": "analyzed"
        }
    
    def execute_task(state: AgentState) -> dict:
        """Execute the task based on analysis."""
        task = state.get("task", "")
        analysis = state.get("analysis", "")
        response = llm.invoke(
            f"Given this task: {task}\n\nAnd this analysis: {analysis}\n\nExecute the task:"
        )
        return {
            "messages": [f"Execution: {response.content}"],
            "result": response.content,
            "status": "executed"
        }
    
    def review_result(state: AgentState) -> dict:
        """Review and finalize the result."""
        result = state.get("result", "")
        response = llm.invoke(f"Review and improve this result if needed:\n\n{result}")
        return {
            "messages": [f"Final: {response.content}"],
            "result": response.content,
            "status": "complete"
        }
    
    # Define routing logic
    def should_continue(state: AgentState) -> Literal["execute", "review", "end"]:
        """Determine next step based on status."""
        status = state.get("status", "")
        if status == "analyzed":
            return "execute"
        elif status == "executed":
            return "review"
        else:
            return "end"
    
    # Build the graph
    workflow = StateGraph(AgentState)
    
    # Add nodes
    workflow.add_node("analyze", analyze_task)
    workflow.add_node("execute", execute_task)
    workflow.add_node("review", review_result)
    
    # Add edges
    workflow.add_edge(START, "analyze")
    workflow.add_conditional_edges(
        "analyze",
        should_continue,
        {"execute": "execute", "review": "review", "end": END}
    )
    workflow.add_conditional_edges(
        "execute",
        should_continue,
        {"execute": "execute", "review": "review", "end": END}
    )
    workflow.add_edge("review", END)
    
    # Compile with checkpointing for state persistence
    memory = MemorySaver()
    app = workflow.compile(checkpointer=memory)
    
    return app

# Build and test the LangGraph workflow
print("Building LangGraph workflow...")
my_workflow = build_langgraph_agent()
print("✅ LangGraph workflow built successfully!")

# Quick test
print("\n--- Quick Test ---")
result = my_workflow.invoke(
    {"task": "Calculate the sum of 1 to 10", "messages": []},
    config={"configurable": {"thread_id": "test-1"}}
)
print(f"Status: {result['status']}")
print(f"Result: {result['result'][:200]}...")

In [ ]:
# TODO: Build your agent using one of the templates above
# or create your own implementation

# my_agent = build_langchain_agent()
# OR
# my_team = build_autogen_team()
# OR
# my_crew = build_crewai_crew()
# OR
# my_workflow = build_langgraph_agent()

## Part 5: Test Your Agent

Run test cases to verify your agent works correctly.

In [ ]:
# Test cases for Code Assistant
code_assistant_tests = [
    "Analyze this code: def add(a, b): return a + b",
    "What does this function do: def factorial(n): return 1 if n <= 1 else n * factorial(n-1)",
    "Suggest improvements for: for i in range(len(lst)): print(lst[i])"
]

# Test cases for Research Agent
research_agent_tests = [
    "Research the benefits of microservices architecture",
    "Find information about Python async programming",
    "What are the latest trends in AI development?"
]

# Test cases for Data Analyst
data_analyst_tests = [
    "Calculate statistics for: 10, 20, 30, 40, 50",
    "Analyze the trend in: 100, 120, 115, 130, 145, 160",
    "Find the average of: 85, 90, 78, 92, 88"
]

print("Test cases ready!")

In [ ]:
# Run tests based on your framework choice

# ===== LANGCHAIN TESTS =====
def test_langchain_agent(agent, tests):
    """Test a LangChain agent with a list of test inputs."""
    results = []
    for i, test in enumerate(tests, 1):
        print(f"\n{'='*50}")
        print(f"Test {i}: {test}")
        print('='*50)
        try:
            result = agent.invoke({"input": test})
            print(f"✅ Result: {result['output']}")
            results.append({"input": test, "output": result['output'], "passed": True})
        except Exception as e:
            print(f"❌ Error: {e}")
            results.append({"input": test, "output": str(e), "passed": False})
    return results

# Example: Test LangChain agent
# results = test_langchain_agent(my_agent, code_assistant_tests)

# ===== AUTOGEN TESTS =====
def test_autogen_team(user_proxy, manager, test_message):
    """Test an AutoGen team with a message."""
    print(f"\n{'='*50}")
    print(f"Test: {test_message}")
    print('='*50)
    try:
        user_proxy.initiate_chat(manager, message=test_message)
        print("✅ AutoGen conversation completed")
        return True
    except Exception as e:
        print(f"❌ Error: {e}")
        return False

# Example: Test AutoGen team
# test_autogen_team(user_proxy, manager, "Research the benefits of Python for data science")

# ===== CREWAI TESTS =====
def test_crewai_crew(crew, inputs):
    """Test a CrewAI crew with inputs."""
    print(f"\n{'='*50}")
    print(f"Test: {inputs}")
    print('='*50)
    try:
        result = crew.kickoff(inputs=inputs)
        print(f"✅ Result: {result}")
        return result
    except Exception as e:
        print(f"❌ Error: {e}")
        return None

# Example: Test CrewAI crew
# test_crewai_crew(my_crew, {"topic": "machine learning best practices"})

# ===== LANGGRAPH TESTS =====
def test_langgraph_workflow(workflow, task, thread_id="test"):
    """Test a LangGraph workflow with a task."""
    print(f"\n{'='*50}")
    print(f"Test: {task}")
    print('='*50)
    try:
        result = workflow.invoke(
            {"task": task, "messages": []},
            config={"configurable": {"thread_id": thread_id}}
        )
        print(f"✅ Status: {result['status']}")
        print(f"✅ Result: {result['result'][:300]}...")
        return result
    except Exception as e:
        print(f"❌ Error: {e}")
        return None

# Example: Test LangGraph workflow
# test_langgraph_workflow(my_workflow, "Analyze the performance of sorting algorithms")

print("Test functions defined! Uncomment the examples above to run tests.")

In [ ]:
# Document your test results
test_results = {
    "test_1": {
        "input": "",
        "expected": "",
        "actual": "",
        "passed": None
    },
    "test_2": {
        "input": "",
        "expected": "",
        "actual": "",
        "passed": None
    },
    "test_3": {
        "input": "",
        "expected": "",
        "actual": "",
        "passed": None
    }
}

print("Document your test results above!")

## Part 6: Prepare Your Demo

Create a demo-ready showcase of your agent.

In [ ]:
# Demo script - customize for your agent

def run_demo():
    """Run a demo of your agent (2-3 minutes)."""
    print("="*60)
    print("🤖 AGENT DEMO: [Your Agent Name]")
    print("="*60)
    
    # 1. Introduction (30 seconds)
    print("\n📋 INTRODUCTION")
    print("-" * 40)
    print("Agent Type: [Code Assistant / Research Agent / Data Analyst / Custom]")
    print("Framework: [LangChain / AutoGen / CrewAI / LangGraph]")
    print("Purpose: [What does your agent do?]")
    
    # 2. Demo Task 1 - Simple example (45 seconds)
    print("\n🔹 DEMO 1: Simple Task")
    print("-" * 40)
    # Uncomment and customize:
    # result = my_agent.invoke({"input": "Your simple task here"})
    # print(f"Result: {result['output']}")
    print("[Run your first demo task]")
    
    # 3. Demo Task 2 - More complex example (45 seconds)
    print("\n🔹 DEMO 2: Complex Task")
    print("-" * 40)
    # Uncomment and customize:
    # result = my_agent.invoke({"input": "Your complex task here"})
    # print(f"Result: {result['output']}")
    print("[Run your second demo task]")
    
    # 4. Key Features Highlight (30 seconds)
    print("\n✨ KEY FEATURES")
    print("-" * 40)
    print("• Feature 1: [e.g., Memory for context retention]")
    print("• Feature 2: [e.g., Multiple specialized tools]")
    print("• Feature 3: [e.g., Error handling and retries]")
    
    # 5. Wrap-up
    print("\n" + "="*60)
    print("🎉 DEMO COMPLETE")
    print("="*60)
    print("\nQuestions?")

# Run your demo
# run_demo()

# ===== QUICK DEMO EXAMPLES =====

# LangChain Demo
def demo_langchain():
    print("🔹 LangChain Agent Demo")
    result = my_agent.invoke({"input": "What is 25 * 4 and save a note about it"})
    print(f"Result: {result['output']}")

# LangGraph Demo  
def demo_langgraph():
    print("🔹 LangGraph Workflow Demo")
    result = my_workflow.invoke(
        {"task": "Explain the difference between lists and tuples in Python", "messages": []},
        config={"configurable": {"thread_id": "demo-1"}}
    )
    print(f"Result: {result['result']}")

# Uncomment to run quick demos:
# demo_langchain()
# demo_langgraph()

## Part 7: Reflection and Documentation

In [ ]:
# Final project documentation
project_summary = {
    "agent_name": "",
    "description": "",
    "framework_used": "",
    "tools_implemented": [],
    "key_features": [],
    "challenges_faced": "",
    "lessons_learned": "",
    "future_improvements": ""
}

print("Fill in your project summary!")

In [ ]:
# Framework comparison reflection
framework_reflection = {
    "framework_used": "",
    "why_good_fit": "",
    "limitations_encountered": "",
    "would_use_different_framework_for": ""
}

print("Reflect on your framework choice!")

## Capstone Checklist

Before presenting, verify:

- [ ] Agent runs without errors
- [ ] At least 2-3 tools are implemented
- [ ] Agent accomplishes its stated goal
- [ ] Demo can be completed in 2-3 minutes
- [ ] Code is reasonably organized
- [ ] You can explain how it works

## Workshop Summary

Congratulations on completing the Crafting Custom Agents Workshop!

### What You Learned

**Session 1:**
- ReAct pattern and agent fundamentals
- LangChain agents with custom tools
- AutoGen conversational multi-agent systems
- CrewAI role-based crews

**Session 2:**
- LangGraph stateful workflows
- Conditional routing and cycles
- Human-in-the-loop patterns
- Framework selection and trade-offs

### Framework Quick Reference

| Framework | Best For |
|-----------|----------|
| LangChain | Single agents with tools |
| AutoGen | Conversational multi-agent |
| CrewAI | Role-based pipelines |
| LangGraph | Complex stateful workflows |

### Keep Learning

- [LangChain Documentation](https://python.langchain.com/docs/)
- [LangGraph Tutorials](https://langchain-ai.github.io/langgraph/tutorials/)
- [AutoGen Documentation](https://microsoft.github.io/autogen/)
- [CrewAI Documentation](https://docs.crewai.com/)

### Thank You!

We hope this workshop has given you the skills to build powerful AI agents for your projects!

## Reference Solution

Need inspiration? A complete reference implementation is available:

**File:** `../../Solutions/lab4_capstone_compliance_agent.py`

This solution demonstrates a **Compliance Document Reviewer** using LangGraph that:
- Classifies document types (email, report, memo, contract)
- Extracts compliance-relevant entities (parties, dates, regulations)
- Flags potential risks with severity levels
- Routes high-risk documents for human review (human-in-the-loop)
- Generates executive summaries and recommendations

**Key Patterns Demonstrated:**
- `TypedDict` state management
- Conditional routing based on risk score
- `interrupt_before` for human approval checkpoints
- JSON parsing with graceful fallbacks

Run it with:
```bash
python Solutions/lab4_capstone_compliance_agent.py
```

> **Note:** Review the reference solution *after* attempting your own implementation to maximize learning.

In [ ]:
print("Workshop Complete!")
print("\nYou are now equipped to build AI agents using multiple frameworks.")
print("Go forth and build amazing things!")